In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import yfinance as yf

import torch
import torch.nn as nn

In [2]:
companyName = 'AMZN'
start_date = '1997-05-15'
end_date = '2023-04-05'
df = yf.download(companyName, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [3]:
df.reset_index(inplace=True)

In [5]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1997-05-15,0.121875,0.125000,0.096354,0.097917,0.097917,1443120000
1,1997-05-16,0.098438,0.098958,0.085417,0.086458,0.086458,294000000
2,1997-05-19,0.088021,0.088542,0.081250,0.085417,0.085417,122136000
3,1997-05-20,0.086458,0.087500,0.081771,0.081771,0.081771,109344000
4,1997-05-21,0.081771,0.082292,0.068750,0.071354,0.071354,377064000
...,...,...,...,...,...,...,...
6510,2023-03-29,98.690002,100.419998,98.559998,100.250000,100.250000,49783300
6511,2023-03-30,101.550003,103.040001,101.010002,102.000000,102.000000,53633400
6512,2023-03-31,102.160004,103.489998,101.949997,103.290001,103.290001,56750300
6513,2023-04-03,102.300003,103.290001,101.430000,102.410004,102.410004,41135700


In [4]:
class PositionalEncodingLayer(nn.Module):
   
   def __init__(self, d_model, max_len=100):
       super(PositionalEncodingLayer, self).__init__()
       self.d_model = d_model
       self.max_len = max_len
   
   def get_angles(self, positions, indexes):
       d_model_tensor = torch.FloatTensor([[self.d_model]]).to(positions.device)
       angle_rates = torch.pow(10000, (2 * (indexes // 2)) / d_model_tensor)
       return positions / angle_rates

   def forward(self, input_sequences):
       """
       :param Tensor[batch_size, seq_len] input_sequences
       :return Tensor[batch_size, seq_len, d_model] position_encoding
       """
       positions = torch.arange(input_sequences.size(1)).unsqueeze(1).to(input_sequences.device) # [seq_len, 1]
       indexes = torch.arange(self.d_model).unsqueeze(0).to(input_sequences.device) # [1, d_model]
       angles = self.get_angles(positions, indexes) # [seq_len, d_model]
       angles[:, 0::2] = torch.sin(angles[:, 0::2]) # apply sin to even indices in the tensor; 2i
       angles[:, 1::2] = torch.cos(angles[:, 1::2]) # apply cos to odd indices in the tensor; 2i
       position_encoding = angles.unsqueeze(0).repeat(input_sequences.size(0), 1, 1) # [batch_size, seq_len, d_model]
       return position_encoding
